In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from math import ceil
import scipy.stats as stats 
%matplotlib inline

In [0]:
d_crime = pd.read_csv("/content/gdrive/My Drive/SF dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")

In [0]:
d_crime.head(2)

In [0]:
#setting x variable and y variable
x = ["X","Y","PdDistrict"]
y = "Category"

In [0]:
#This function transforms the categorical y into numeric, and prepare it for multiclass classfication
def transformation(dataset,y,x):
    for i in dataset.columns:
        if i not in x and i != y:
            dataset.drop(i,axis=1,inplace=True)         #drop unrelavent columns
    dataset.dropna(axis=0, how='any', inplace=True)    
    neighborhood=pd.get_dummies(dataset["PdDistrict"])
    dataset=pd.concat([dataset,neighborhood],axis=1)
    dataset.drop("PdDistrict",axis=1,inplace=True)
    y_variable = dataset[y].unique()
    dic_y = {}                       # use a dictionary to temporarily hold crime data, transform crime into numeric representation
    i = 1 
    for crime_type in y_variable:         
        dic_y[crime_type]=i
        i=i+1
    dataset.replace(to_replace=dic_y,inplace=True)
    return dataset,dic_y

df,temp = transformation(d_crime,y,x)
df,temp

In [0]:
sns.countplot(x="Category",data=df)  #shows the number of crimes in numeric representation

In [0]:
# I dropped Y variable for X, and split dataset 0.3 vs 0.7
X=df.drop("Category",axis=1)
Y=df["Category"]
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3)

In [0]:
#build a multinomial logistic model
log_model = LogisticRegression().fit(x_train,y_train)
prediction = log_model.predict(x_test)
prediction

In [0]:
#Evaluate the model based on prediction accuracy
model_score = accuracy_score(y_test,prediction)
model_score  

In [0]:
#predict the log-probability of the sample for each class/crime
proba = log_model.predict_proba(x_test)

This table shows the likelihood of each crime happening on that geo cordinates

In [0]:
#this function takes as input a dictionary, and return the key
def getback_columns(dictionary,number):
    for key,value in dictionary.items():
        if value==number:
            return key
        else:
            continue
column_names = []       # this list holds column names so will be used later to construct the output table
for classes in log_model.classes_:
    key = getback_columns(temp,classes)
    column_names.append(key)
column_names
output_table = pd.DataFrame(proba, columns=column_names)
output_table

In [0]:
#Evaluating the model with precision, recall, and f1 score
print(classification_report(y_test,prediction))

In [0]:
#Evaluate the model with confusion matrix
matrix = confusion_matrix(y_test,prediction )

In [0]:
#Put confusion matrix data into a dataframe, and visualize it with heatmap
confusion_df = pd.DataFrame(matrix,
                     index = temp.keys(), 
                     columns = temp.keys())
plt.figure(figsize=(30,20))
plt.title('Multinomial logistic regression\nAccuracy:{0:.3f}'.format(model_score))
plt.ylabel('True label')
plt.xlabel('Predicted label')
sns.heatmap(confusion_df, annot=True)
plt.show()

Reference list:
1. https://dataaspirant.com/2017/03/14/multinomial-logistic-regression-model-works-machine-learning/